In [1]:
from itertools import combinations
from hashlib import sha256
from nltk import ngrams
from struct import pack
from random import choices

# PPRL with record-level Bloom filters
For a Bloom filter we define:

- a bit vector $v$ of length $l$ with all values initially set to $0$;
- a set $H$ with $k$ independent hash functions with range $0$ to $l - 1$.

To convert a set $S$ to a Bloom filter:

- each element $x_i \in S$ is hashed using the $k$ hash functions;
- all bits in $v$ with indices $h_j(x_i)$, for $1 \leq j \leq k$, are set to $1$;

First, all fields are converted to a set of n-grams. This n-gram set is used to create field-level Bloom filters (FBFs).

In [2]:
l = 512
k = 2
H = [sha256(sha256(bytes(i)).digest()) for i in range(k)]

def n_grams(field):
    """Converts a field to a set of n-grams."""
    return [''.join(ng) for ng in ngrams(' {} '.format(field), 2)]

def bit_vector(size):
    """Returns a bit vector with all values set to zero."""
    return [0 for _ in range(size)]

def hash_indices(x):
    """Returns the indices generated by h(x)."""
    if type(x) is str:
        x = x.encode('UTF-8')
    
    for h in H:
        g = h.copy()
        g.update(x)
        
        digest = int(g.hexdigest(), 16)
        yield digest % l

In [3]:
p1 = ('John', 'Smith', 'Amsterdam')
p2 = ('John', 'Smyth', 'Amsterdam')
p3 = ('Johhny', 'Smith', 'Den Haag')

p1_S = [n_grams(f) for f in p1]
p2_S = [n_grams(f) for f in p2]
p3_S = [n_grams(f) for f in p3]

p1_v = [bit_vector(l) for _ in p1]
p2_v = [bit_vector(l) for _ in p2]
p3_v = [bit_vector(l) for _ in p3]

all_p = [p1] + [p2] + [p3]
all_S = p1_S + p2_S + p3_S
all_v = p1_v + p2_v + p3_v

# Construct FBFs.
for (S, v) in zip(all_S, all_v):
    for x in S:
        for i in hash_indices(x):
            v[i] = 1

Uniquely to each field we randomly, with replacement, genereate indices that mark the positions of the FBF bits that will be used to create record-level Bloom filters (RBFs). For this we define:

- $w_i$ as the weight (percentage) for each field $i$;
- $m$ as the number of bits to include in the RBF;

The amount of bits to draw from the $i$-th FBF is $\frac{w_i}{100}m$.




In [4]:
w1 = 25
w2 = 35
w3 = 40
m = 1280

def draw_bits(w, m, all_fbf):
    amount = int((w / 100) * m)
    indices = choices(range(l), k=amount)
    
    bits = []
    for fbf in all_fbf:
        bits.append([fbf[i] for i in indices])
        
    return bits

In [5]:
all_w = [w1, w2, w3]
all_rbf = [[], [], []]

for (w, a, b, c) in zip(all_w, p1_v, p2_v, p3_v):
    for i, bits in enumerate(draw_bits(w, m, [a, b, c])):
        all_rbf[i] += bits

Similarity between RBFs is calculated using the Dice-coefficient:

$$Dice\_sim(v_1, v_2) = \frac{2c}{x_1 + x_2}$$

- $c$ is the number of common bit positions in $v_1$ and $v_2$ that are set to $1$;
- $x_1$ is the number of bit positions in $v_1$ set to $1$;
- $x_2$ is the number of bit positions in $v_2$ set to $1$.

In [6]:
def dice_sim(a, b):
    c = sum([1 for (i, j) in zip(a, b) if i + j == 2])
    x1 = sum(a)
    x2 = sum(b)
        
    return (2*c) / (x1 + x2)

for (a, b) in combinations(zip(all_p, all_rbf), 2):
    record_a = ' '.join(a[0])
    record_b = ' '.join(b[0])
    similarity = dice_sim(a[1], b[1])
    
    print('Similarity between \'{0}\' and \'{1}\': {2:.3}'.format(record_a, record_b, similarity))

Similarity between 'John Smith Amsterdam' and 'John Smyth Amsterdam': 0.909
Similarity between 'John Smith Amsterdam' and 'Johhny Smith Den Haag': 0.4
Similarity between 'John Smyth Amsterdam' and 'Johhny Smith Den Haag': 0.314


## References
- [Privacy-preserving record linkage using Bloom filters](https://www.semanticscholar.org/paper/Privacy-preserving-record-linkage-using-Bloom-filt-Schnell-Bachteler/8392fa13e5013073b617e947b0229bf1734990ac)
- [Composite Bloom filters for secure record linkage](https://www.semanticscholar.org/paper/Composite-Bloom-Filters-for-Secure-Record-Linkage-Durham-Kantarcioglu/bb1dbac86d922b91daabf61ec75f56dde47997be)